In [63]:
import pandas as pd

train_df = pd.read_csv("../data/test_task_training_dataset.csv", index_col=0)
hold_out_df = pd.read_csv("../data/test_task_live_dataset.csv", index_col=0)
dataset = pd.read_excel("../data/test_task_dataset.xlsx")

train_df.head()

,loan_id,loan_created_on,loan_amount,loan_term,loan_due,loan_paid,loan_paid_on,loan_issued,client_id,client_birth_date,client_income,client_expenses,client_liabilities,client_debt,loan_is_matured,loan_fully_paid,number_of_previous_loans
0,1,6/15/2019,900,69,1183,1183.0,8/23/2019,1,1,12/31/1981,1150,595,0,0,True,1,0
1,2,5/23/2019,700,90,935,935.0,8/21/2019,1,2,9/10/1998,658,556,0,837,True,1,0
2,6,5/29/2019,1100,30,1407,1407.0,6/28/2019,1,6,1/30/1973,2301,857,1616,50,True,1,0
3,7,6/2/2019,1500,24,1834,NaN,NaN,1,7,11/22/1986,1494,751,2738,0,True,0,0
4,8,5/24/2019,200,30,256,256.0,6/23/2019,1,8,11/13/1986,3196,208,0,0,True,1,0


In [3]:
train_df.shape

(62952, 17)

In [4]:
train_df.loan_issued.unique()

array([1])

In [4]:
hold_out_df.head()

,loan_id,loan_created_on,loan_amount,loan_term,loan_due,loan_issued,client_id,client_birth_date,client_income,client_expenses,client_liabilities,client_debt,number_of_previous_loans
0,99452,2/21/2021,1500,22,1807,1,65652,9/24/1980,3934,795,50,0,0
1,99454,2/6/2021,800,30,1023,1,65654,5/22/2001,2047,104,0,0,0
2,99460,2/12/2021,1100,28,1386,1,65660,8/31/1983,6469,109,50,0,0
3,99461,3/3/2021,900,22,1084,1,65661,7/28/1993,2367,1243,0,0,0
4,99462,3/5/2021,1600,24,1956,1,65662,10/17/1978,12942,18,50,0,0


In [5]:
hold_out_df.shape

(10730, 13)

In [6]:
hold_out_df.loan_issued.unique()

array([1])

In [7]:
list(set(train_df.columns) - set(hold_out_df.columns))

['loan_fully_paid', 'loan_paid_on', 'loan_is_matured', 'loan_paid']

In [8]:
dataset

,loan_id,"unique ID of loan application. If issued, then also represents a unique ID of a loan."
0,created_on,date on which loan application was created. I...
1,loan_amount,loan amount requested by the client. If issue...
2,loan_term,loan term (in days) requested by the client. ...
3,loan_due,total amount that client is expected to repay...
4,loan_paid,total amount repaid by the client for this loan.
5,loan_paid_on,date on which client made a repayment.
6,loan_issued,"1 if this loan was issued to the client, 0 ot..."
7,client_id,unique ID of client.
8,client_birth_date,client's birth date.
9,client_income,client's verified monthly net income at the m...


# Part 1. Dataset exploration and developing hypotheses

Value to predict: loan_fully_paid.
- The 'loan_fully_paid' flag is measured based on the following condition: **1 if loan_paid/loan_due >= 1, otherwise 0**
### a.1) Variables that can be safely used for credit scoring
- client_birth_date and created_on - for calculating the age of the client and filtering out observations with underage clients
- loan_amount
- loan_term
- loan_due
- client_income
- client_expenses 
- client_liabilities
- client_income
- client_debt
- number_of_previous_loans

### a.2) Variables that leak data about the client performance past when the final underwriting decision would be made.
- Columns **'loan_paid'**, **'loan_paid_on'** shouldn't be taken into consideration while training the model, as those values would be unknown in real environment and they are related to the **'loan_fully_paid'** feature (feature to predict). These variables should be calculated based on the client behaviour after the final decision is made (whether the loan is issued to the client or not).
- Column **'loan_issued'** in reality would be unknown and affected by the final decision would be made based on credit risk evaluation. For example, the decision whether the loan should be issued to the client or not might depend on the prediction of the feature that represents whether the loan would be fully paid or not. Current training and testing datasets have all customers with loan earlier issued.
- Column **'loan_is_matured'** also leaks the performance that is past the final decision made whether to issue the loan to a client or not.
### a.3) What clients should be filtered from the dataset?
- Clients whose age at the date of submitting the loan application is less than 18 years.

### b) Variables that are expected to be useful
- loan_amount
- loan_term
- loan_due
- client_income
- client_expenses 
- client_liabilities
- client_income
- client_debt
- number_of_previous_loans
- age calculated as the difference in years between the client_birth_date and created_on features

### c) What other variables / client information would have been interesting to see and you would have liked to have been added to the dataset?
- Number of applications currently submitted by the client
- Information regarding the client citizenship and residence for filtering out clients based on regulations
- Number of previous defaults in this institution
- Employment status
- Is a client currently on probation period at work or not? Might be suitable for issuing loan with bigger amounts
- Number of previous loans issued by other instituations (if access to this data is available)
- Number of previous defaults in other institutions (if access to this data is available)
- Purpose of the requested loan (how would the client like to spend this money?)
- Purposes of previous requested loans
- Gender
- Marital status

# Part 2. Model for credit risk assessment

In [9]:
train_df.describe()

,loan_id,loan_amount,loan_term,loan_due,loan_paid,loan_issued,client_id,client_income,client_expenses,client_liabilities,client_debt,loan_fully_paid,number_of_previous_loans
count,62952.000000,62952.000000,62952.000000,62952.000000,53333.000000,62952.0,62952.000000,62952.000000,62952.000000,62952.000000,62952.000000,62952.000000,62952.000000
mean,51001.340529,1127.792604,43.653911,1445.927024,1417.606968,1.0,29092.998173,3026.524701,638.005703,185.678183,2250.456364,0.847201,0.595819
std,28630.753764,740.854810,26.114295,965.540929,964.017991,0.0,17984.749660,1955.224262,676.195292,488.275787,13729.679686,0.359796,1.041955
min,1.000000,100.000000,5.000000,105.000000,105.000000,1.0,1.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26296.750000,600.000000,27.000000,767.000000,668.000000,1.0,13964.000000,1612.000000,162.000000,0.000000,0.000000,1.000000,0.000000
50%,51298.000000,1000.000000,30.000000,1269.000000,1251.000000,1.0,27907.500000,2605.000000,423.000000,0.000000,0.000000,1.000000,0.000000
75%,76038.250000,1500.000000,74.000000,1903.000000,1876.000000,1.0,43529.500000,3977.000000,882.000000,88.000000,50.000000,1.000000,1.000000
max,99451.000000,5000.000000,90.000000,6682.000000,7962.000000,1.0,65650.000000,19516.000000,8218.000000,14260.000000,450626.000000,1.000000,11.000000


In [64]:
def clean_dataset(df):
    res = df.copy()
    cols_to_drop = ['loan_paid', 'loan_paid_on', 'loan_is_matured']
    if set(cols_to_drop).issubset(res.columns):
        res = res.drop(cols_to_drop, axis=1)
    res['age'] = round((pd.to_datetime(res['loan_created_on']) - pd.to_datetime(res['client_birth_date'])).dt.days / 365, 1)
    res = res[res['age'] >= 18]
    res.drop(['loan_created_on', 'client_birth_date', 'client_id', 'loan_issued'], axis=1, inplace=True)
    res.set_index('loan_id', inplace=True)
    return res

cleaned_train_df = clean_dataset(train_df)
cleaned_train_df.head()

,loan_amount,loan_term,loan_due,client_income,client_expenses,client_liabilities,client_debt,loan_fully_paid,number_of_previous_loans,age
loan_id,,,,,,,,,,
1,900,69,1183,1150,595,0,0,1,0,37.5
2,700,90,935,658,556,0,837,1,0,20.7
6,1100,30,1407,2301,857,1616,50,1,0,46.4
7,1500,24,1834,1494,751,2738,0,0,0,32.5
8,200,30,256,3196,208,0,0,1,0,32.5


In [65]:
cleaned_train_df.describe()

,loan_amount,loan_term,loan_due,client_income,client_expenses,client_liabilities,client_debt,loan_fully_paid,number_of_previous_loans,age
count,57857.000000,57857.000000,57857.000000,57857.000000,57857.000000,57857.000000,57857.000000,57857.000000,57857.000000,57857.000000
mean,1147.610488,43.769604,1471.515737,3036.306895,639.934908,187.632473,2241.401231,0.853086,0.631315,33.943896
std,751.061336,26.164679,978.986398,1962.159030,678.464853,492.084206,13718.279718,0.354023,1.072543,11.049441
min,100.000000,5.000000,105.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000
25%,700.000000,27.000000,849.000000,1615.000000,161.000000,0.000000,0.000000,1.000000,0.000000,25.600000
50%,1000.000000,30.000000,1279.000000,2615.000000,424.000000,0.000000,0.000000,1.000000,0.000000,32.100000
75%,1500.000000,74.000000,1918.000000,3992.000000,886.000000,90.000000,50.000000,1.000000,1.000000,40.400000
max,5000.000000,90.000000,6682.000000,18244.000000,8218.000000,14260.000000,360131.000000,1.000000,11.000000,98.600000


In [66]:
cleaned_train_df.duplicated().any()

False

In [67]:
len(train_df.loan_id.unique())

62952

In [68]:
train_df.shape

(62952, 17)

In [69]:
len(train_df.client_id.unique())

40638

In [70]:
cleaned_train_df.isna().sum()

loan_amount                 0
loan_term                   0
loan_due                    0
client_income               0
client_expenses             0
client_liabilities          0
client_debt                 0
loan_fully_paid             0
number_of_previous_loans    0
age                         0
dtype: int64

In [71]:
cleaned_train_df['loan_fully_paid'].value_counts()

1    49357
0     8500
Name: loan_fully_paid, dtype: int64

In [3]:
cleaned_train_df['loan_fully_paid'].value_counts() / float(len(cleaned_train_df))

1    0.853086
0    0.146914
Name: loan_fully_paid, dtype: float64

In [54]:
cleaned_train_df.shape

(57857, 10)

# Model 1: random forest classifier on unbalanced data

In [28]:
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np


scaler = StandardScaler()

X = cleaned_train_df[cleaned_train_df.columns.difference(['loan_fully_paid'])]
y = cleaned_train_df['loan_fully_paid'].values

clf = RandomForestClassifier()
scoring = ['balanced_accuracy', 'precision_weighted', 'recall_weighted', 'roc_auc', 'f1_weighted']
steps = [('transformer', scaler), ('estimator', clf)]
pipeline = Pipeline(steps=steps)

cv = KFold(n_splits=5)
cv_scores = cross_validate(pipeline, X, y, scoring=scoring, cv=cv, return_estimator=True)

In [31]:
np.mean(cv_scores['test_precision_weighted'])

0.8328513343227716

In [32]:
np.mean(cv_scores['test_recall_weighted'])

0.8610029340488305

In [30]:
np.mean(cv_scores['test_balanced_accuracy'])

0.5688018650500284

In [29]:
np.mean(cv_scores['test_roc_auc'])

0.7310341345780562

In [33]:
np.mean(cv_scores['test_f1_weighted'])

0.8238334360698975

## Model 2: random forest classifier on data over-sampled using SMOTE

In [56]:
scaler = StandardScaler()

X = cleaned_train_df[cleaned_train_df.columns.difference(['loan_fully_paid'])]
y = cleaned_train_df['loan_fully_paid'].values

clf = RandomForestClassifier()
scoring = ['accuracy', 'precision', 'recall', 'roc_auc', 'f1']
steps = [('transformer', scaler), ('over', SMOTE()), ('estimator', clf)]
pipeline = Pipeline(steps=steps)

cv = KFold(n_splits=5)
cv_scores = cross_validate(pipeline, X, y, scoring=scoring, cv=cv, return_estimator=True)

In [21]:
np.mean(cv_scores['test_precision'])

0.8834928447452226

In [22]:
np.mean(cv_scores['test_recall'])

0.9093770539472977

In [23]:
np.mean(cv_scores['test_accuracy'])

0.8196601301578081

In [19]:
np.mean(cv_scores['test_roc_auc'])

0.7194437773176107

In [20]:
np.mean(cv_scores['test_f1'])

0.8946390541382326

# Part 3.  Results evaluation
1. As it might be seen, the model nr. 1 fitted to the imbalanced data that is strongly skewed towards the default, has a bit low balanced accuracy that does not outperform random guessing (56,88%). However, this model would not misclassify non-default related observations as false positives with 83,28% level of confidence.

2. The model nr. 2 that was fitted to the over-sampled data outperforms in terms of accuracy (81,97%), precision, and recall. However, accuracy is not the best metric when dealing with imbalanced data. Interestingly, the AUC is not strongly affected by balancing the data (see 'test_roc_auc' values above). This is the modelthat is used to make predictions on the hold-out dataset. This model is expected to get relatively low number of false positives (precision score 88%). However, over-sampling brings the risk of over-fitting and might lead to the lower predictive performance in production.

Note: over-sampling option was chosen because the under-sampling would lead to losing the potentially representative observations.

# Part 4. Prediction on the live dataset


In [72]:
test_df = clean_dataset(hold_out_df)
test_df.duplicated().any()

False

In [80]:
test_df.head()

,age,client_debt,client_expenses,client_income,client_liabilities,loan_amount,loan_due,loan_term,number_of_previous_loans
loan_id,,,,,,,,,
99452,40.4,0,795,3934,50,1500,1807,22,0
99454,19.7,0,104,2047,0,800,1023,30,0
99460,37.5,0,109,6469,50,1100,1386,28,0
99461,27.6,0,1243,2367,0,900,1084,22,0
99462,42.4,0,18,12942,50,1600,1956,24,0


In [73]:
test_df.isna().sum()

loan_amount                 0
loan_term                   0
loan_due                    0
client_income               0
client_expenses             0
client_liabilities          0
client_debt                 0
number_of_previous_loans    0
age                         0
dtype: int64

In [74]:
test_df.shape

(9949, 9)

In [78]:
from sklearn.pipeline import Pipeline


scaler = StandardScaler()
clf = RandomForestClassifier()

steps = [('transformer', scaler), ('estimator', clf)]
pipeline = Pipeline(steps=steps)


pipeline.fit(X, y)
test_df = test_df[X.columns.values] #re-arrange the order of columns
predictions = pipeline.predict(test_df)
pd.DataFrame(predictions).to_csv("../data/predictions.csv")

In [81]:
pd.DataFrame(predictions).value_counts()

1    9591
0     358
dtype: int64

# Part 5. Conclusion

a) Next steps might include collecting more observations that belong to the default segment (clients who failed to fully pay the loan in time). Moreover, it might be good to keep the client id variable that was not taken into account in this model and aggregate observations that belong to the client with the same id. It might be good to compare several models with the dummy baseline model in terms of AUC, F1 score, precision, and recall. Accuracy metric will be helpful if it is possible to collect balanced dataset. If choose to proceed with imbalanced dataset, then might be possible to combine under-sampling and over-sampling techniques for achieving the better results (as an example, apply under-sampling of the majority class with the target fraction of choice, and after that apply SMOTE or other improved variants of SMOTE). Another aproach that might be used is unsupervised learning based, for example, clustering. Moreover, it might be helpful to check the correlation of variables before building the model.

b) The benefit of the current model lies in relatively high distinguishing power of the model. According to cross-validation results on the training data set, current model is able to distinguish default and non-default observations with the probability 72% (see test_ROC_AUC). At the same time, high precision score (88%) indicates that there are less false positives (observations related to loan-fully-paid class detected as defaults) are expected.

c) Next steps for implementing this model to improve the business will include improving the quality of the model with the aim to achieve higher predictive power (ROC-AUC results indicate that the model will distinguish defaults with probability around 72%, so this model still needs improvement). One of the approaches is collecting more default-related observations. Another approach might be to engineer more features (possible options mentioned in part 1). Moreover, it might be possible to predict another metric (example: loan_paid_on - the date when the loan would be paid). All experiments and their outputs should be tracked (example: tracked experiments in MLflow with metrics values, logged models, etc.).